# Performance Attribution
In this code I show the risk attribution of the portfolio returns from the previous strategy. I download the FF3 factors from Kenneth French's website: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html And use them to calculate my factor exposures.

In [59]:
import pandas as pd
import statsmodels.api as sm

In [60]:
ff_3 = pd.read_csv('data/F-F_Research_Data_Factors.CSV', skiprows=3)
ff_3 = ff_3.iloc[:-1]
ff_3.columns = ['date', 'mkt_excess', 'smb', 'hml', 'rf']
ff_3[['mkt_excess', 'smb', 'hml', 'rf']] = ff_3[['mkt_excess', 'smb', 'hml', 'rf']].astype(float) / 100
ff_3 = ff_3[ff_3['date'].astype(str).str.match(r'^\d{6}$')]
ff_3['date'] = pd.to_datetime(ff_3['date'], format='%Y%m')
ff_3 = ff_3.set_index('date')
ff_3.head()

,mkt_excess,smb,hml,rf
date,,,,
1926-07-01,0.0296,-0.0256,-0.0243,0.0022
1926-08-01,0.0264,-0.0117,0.0382,0.0025
1926-09-01,0.0036,-0.0140,0.0013,0.0023
1926-10-01,-0.0324,-0.0009,0.0070,0.0032
1926-11-01,0.0253,-0.0010,-0.0051,0.0031


In [61]:
ew_rets = pd.read_parquet('data/ew_long_short_returns.parquet')
#Add 1 day to the index to match the date format
ew_rets.index = pd.to_datetime(ew_rets.index) + pd.DateOffset(1)
ew_rets = ew_rets.merge(ff_3, left_index=True, right_index=True)
ew_rets = ew_rets[ew_rets['mthret'] != 0]

vw_rets = pd.read_parquet('data/vw_long_short_returns.parquet')
vw_rets.index = pd.to_datetime(vw_rets.index) + pd.DateOffset(1)
vw_rets = vw_rets.merge(ff_3, left_index=True, right_index=True)
vw_rets = vw_rets[vw_rets['weighted_returns'] != 0]

In the regressions below we find that the portfolio does not have statistically significant exposures to the FF3 or the market. The strategy is independent of these risk factors,
but the returns are so poor we end up with an alpha of 0.000074%. The framework has been built here, and can support future work where we may choose to change portfolio weight based on a mandate that requires a certain level of exposure to certain factors.

In [62]:
#Run the regression for the equal weighted returns attribution
regression_df = ew_rets.dropna(subset=['mkt_excess', 'smb', 'hml', 'mthret']).reset_index()
X = sm.add_constant(regression_df[['mkt_excess', 'smb', 'hml']])
y = regression_df['mthret'].astype(float)
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 mthret   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.184
Date:                Wed, 13 Nov 2024   Prob (F-statistic):              0.315
Time:                        04:53:13   Log-Likelihood:                 1481.5
No. Observations:                 661   AIC:                            -2955.
Df Residuals:                     657   BIC:                            -2937.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7.357e-05      0.001      0.072      0.942      -0.002       0.002
mkt_excess    -0.0406      0.023     -1.752      0.080      -0.086       0.005
smb            0.0299      0.034      0.872      0.384      -0.037       0.097
hml            0.0102      0.034      0.304      0.761      -0.056       0.076
==============================================================================
Omnibus:                      135.220   Durbin-Watson:                   2.097
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1141.657
Skew:                           0.648   Prob(JB):                    1.24e-248
Kurtosis:                       9.307   Cond. No.                         35.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
regression_df = vw_rets.dropna(subset=['mkt_excess', 'smb', 'hml', 'weighted_returns']).reset_index()
X = sm.add_constant(regression_df[['mkt_excess', 'smb', 'hml']])
y = regression_df['weighted_returns'].astype(float)
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       weighted_returns   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.4842
Date:                Wed, 13 Nov 2024   Prob (F-statistic):              0.693
Time:                        04:53:13   Log-Likelihood:                 1299.6
No. Observations:                 661   AIC:                            -2591.
Df Residuals:                     657   BIC:                            -2573.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.009e-05      0.001     -0.008      0.994      -0.003       0.003
mkt_excess    -0.0343      0.031     -1.122      0.262      -0.094       0.026
smb           -0.0033      0.045     -0.072      0.942      -0.092       0.085
hml           -0.0206      0.044     -0.465      0.642      -0.108       0.066
==============================================================================
Omnibus:                       73.585   Durbin-Watson:                   2.066
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              515.819
Skew:                           0.137   Prob(JB):                    9.80e-113
Kurtosis:                       7.319   Cond. No.                         35.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""